In [ ]:
from scipy import signal
from scipy.fftpack import fft
import uconnrcmpy as ucr
import numpy as np
from pathlib import Path
from subprocess import run
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1.inset_locator import zoomed_inset_axes, mark_inset, BboxConnector

# %matplotlib

In [ ]:
plt.rc('font', family='serif', serif='Times New Roman', size=22.0)
plt.rc('mathtext', fontset='stix')
dpi = '300'

# Draw the Raw Voltage Trace figure

In [ ]:
volt_trace = ucr.traces.VoltageTrace(Path('10_in_00_mm_333K-979t-100x-13-Jul-15-1349.txt'))
raw_color = 'blue'
filt_color = 'limegreen'
sm_color = 'red'
raw_fig, raw_ax = plt.subplots(figsize=[14, 9])
raw_ax.plot(volt_trace.time*1000, volt_trace.signal[:, 1], label="Original", color=raw_color)
raw_ax.plot(volt_trace.time*1000, volt_trace.filtered_voltage, label="Filtered", color=filt_color)
raw_ax.plot(volt_trace.time*1000, volt_trace.smoothed_voltage, label="Filtered & Smoothed", color=sm_color)
raw_ax.set_xlim([160, 240])
raw_ax.set_ylim([-0.01, 1.01])
raw_ax.set_xlabel('Time [ms]')
raw_ax.set_ylabel('Voltage [V]')
raw_ax_legend = raw_ax.legend(bbox_to_anchor=(0., 1.02, 1., .102), loc=3,
                              ncol=3, mode="expand", borderaxespad=0.)

for leg in raw_ax_legend.legendHandles:
    leg.set_linewidth(2.0)
    
raw_ax.axhline(y=0, color="grey")

# Create the zoomed inset
raw_inset = zoomed_inset_axes(raw_ax, 35, loc=6, bbox_to_anchor=raw_ax.transData.transform((165, 0.6)))
raw_inset.plot(volt_trace.time*1000, volt_trace.signal[:, 1], color=raw_color)
raw_inset.plot(volt_trace.time*1000, volt_trace.filtered_voltage, linewidth=3.0, color=filt_color)
raw_inset.plot(volt_trace.time*1000, volt_trace.smoothed_voltage, color=sm_color)
raw_inset.set_xlim([191.5, 193.25])
raw_inset.set_ylim([0.278, 0.29])
xticks = raw_inset.get_xticks()
raw_inset.set_xticks(xticks[1:-1])
raw_inset.annotate('(a)', (191.55, 0.2885), backgroundcolor='white')

pp, p1, p2 = mark_inset(raw_ax, raw_inset, loc1=3, loc2=4, fc="none", ec="0.5")

zero_inset = zoomed_inset_axes(raw_ax, 41, loc=3, bbox_to_anchor=raw_ax.transData.transform((161.55, 0.05)))
zero_inset.plot(volt_trace.time*1000, volt_trace.signal[:, 1], color=raw_color)
zero_inset.plot(volt_trace.time*1000, volt_trace.filtered_voltage, linewidth=3.0, color=filt_color)
zero_inset.plot(volt_trace.time*1000, volt_trace.smoothed_voltage, color=sm_color)
zero_inset.axhline(y=0, color="grey")
zero_inset.ticklabel_format(style='sci', axis='y', scilimits=(-2, 3), useOffset=False)
zero_inset.set_xlim([161.95, 162.5])
zero_inset.set_ylim([-3.0E-3, 1.0E-3])
zero_inset.set_xticks([162, 162.2])
zero_inset.set_xticklabels(['162', '162.2'])
zero_inset.set_yticks([-3.0E-3, -2.0E-3, -1.0E-3, 0.0, 1.0E-3])
zero_inset.annotate('(b)', (162.4, 0), backgroundcolor='white')

pp2, p3, p4 = mark_inset(raw_ax, zero_inset, loc1=3, loc2=4, fc="none", ec="0.5")

fig_name = 'raw-voltage'

raw_fig.savefig(fig_name + '.pdf', bbox_inches='tight')
run(['convert', '-density', dpi, fig_name + '.pdf', fig_name + '.png'])

# This gets drawn in front of the inset axes, when it should be behind for the proper effect :(
# p3 = BboxConnector(raw_inset.bbox, pp.bbox, loc1=2, loc2=2, fc="none", ec="0.5")
# raw_inset.add_patch(p3)
# p3.set_clip_on(False)

# Plot the 2-stage ignition case

In [ ]:
expt = ucr.dataprocessing.Experiment(Path('00_in_02_mm_373K-1282t-100x-19-Jul-15-1633.txt'))
nr_expt = ucr.dataprocessing.Experiment(Path('NR_00_in_02_mm_373K-1278t-100x-19-Jul-15-1652.txt'))

In [ ]:
ign_fig, pressure_ax = plt.subplots(figsize=[14, 9])
pressure_ax.plot(expt.pressure_trace.zeroed_time*1000,
                 expt.pressure_trace.pressure,
                 linewidth=1.5, label="Pressure")
pressure_ax.plot(nr_expt.pressure_trace.zeroed_time*1000,
                 nr_expt.pressure_trace.pressure,
                 linewidth=1.5, label="Non-Reactive Pressure")
dpdt_ax = pressure_ax.twinx()
dpdt_ax.plot(expt.pressure_trace.zeroed_time*1000,
             expt.pressure_trace.smoothed_derivative/1000,
             'r', linewidth=1.5, label="Time Derivative of Pressure")

dpdt_ax.set_ylim(-0.25, 3)
dpdt_ax.set_ylabel('Time Derivative of Pressure [bar/ms]')

pressure_ax.axvline(x=0, color='k', ymax=0.52, linewidth=1.5)
pressure_ax.axvline(x=expt.ignition_delay, color='k', linewidth=1.5)
pressure_ax.axvline(x=expt.first_stage, color='k', ymax=0.25, linewidth=1.5)
pressure_ax.annotate('', xy=(0, 10), xycoords='data', xytext=(expt.first_stage, 10), textcoords='data',
                     arrowprops={'arrowstyle': '<|-|>', 'shrinkA': 0, 'shrinkB': 0, 'fc': 'black', 'linewidth': 1.5})
pressure_ax.annotate('$τ_1$', xy=(expt.first_stage/2, 10), xycoords='data', xytext=(0, 10), textcoords='offset points')
pressure_ax.annotate('$τ$', xy=(expt.ignition_delay/2, 20), xycoords='data', xytext=(0, 10), textcoords='offset points')
pressure_ax.annotate('', xy=(0, 20), xycoords='data', xytext=(expt.ignition_delay, 20), textcoords='data',
                     arrowprops={'arrowstyle': '<|-|>', 'shrinkA': 0, 'shrinkB': 0, 'fc': 'black', 'linewidth': 1.5})
pressure_ax.annotate('EOC', xy=(0, 31.5), xycoords='data')
pressure_ax.annotate('', xy=(-4, 15), xycoords='data', xytext=(-17, 15), textcoords='data',
                     arrowprops={'arrowstyle': '<|-', 'shrinkA': 0, 'shrinkB': 0, 'fc': 'black', 'linewidth': 1.5})
dpdt_ax.annotate('', xy=(-1, 1.75), xycoords='data', xytext=(12, 1.75), textcoords='data',
                 arrowprops={'arrowstyle': '<|-', 'shrinkA': 0, 'shrinkB': 0, 'fc': 'black', 'linewidth': 1.5})
pressure_ax.set_ylim(0, 60)
pressure_ax.set_xlim(-30, 70)
pressure_ax.set_xlabel('Time [ms]')
pressure_ax.set_ylabel('Pressure [bar]')

lin_1, leg_1 = pressure_ax.get_legend_handles_labels()
lin_2, leg_2 = dpdt_ax.get_legend_handles_labels()
dpdt_ax_legend = dpdt_ax.legend(lin_1 + lin_2, leg_1 + leg_2, bbox_to_anchor=(0.85, 0.95))

for leg in dpdt_ax_legend.legendHandles:
    leg.set_linewidth(2.0)

fig_name = 'ign-delay-def'

ign_fig.savefig(fig_name + '.pdf', bbox_inches='tight')
run(['convert', '-density', dpi, fig_name + '.pdf', fig_name + '.png'])
# pressure_inset = zoomed_inset_axes(pressure_ax, 4, loc=9)
# pressure_inset.plot(expt.pressure_trace.zeroed_time*1000, expt.pressure_trace.pressure)
# pressure_inset.set_xlim(30, 40)
# pressure_inset.set_ylim(27, 30)

# dpdt_inset = pressure_inset.twinx()
# dpdt_inset.plot(expt.pressure_trace.zeroed_time*1000, expt.pressure_trace.smoothed_derivative/1000, 'g')
# # dpdt_inset.set_ylim(0, 5)


# Draw the frequency spectrum

In [ ]:
volt_trace = ucr.traces.VoltageTrace(Path('10_in_00_mm_333K-979t-100x-13-Jul-15-1349.txt'))
samp_freq = np.ceil(1/volt_trace.time[1])

freq_fig, freq_ax = plt.subplots(figsize=[14, 9])    
f, orig_den = signal.welch(volt_trace.signal[:, 1], fs=samp_freq, nperseg=2**14)
freq_ax.plot(f, orig_den, label='Original')
f, filt_den = signal.welch(volt_trace.filtered_voltage, fs=samp_freq, nperseg=2**14)
freq_ax.plot(f, filt_den, label='Filtered')
f, sm_den = signal.welch(volt_trace.smoothed_voltage, fs=samp_freq, nperseg=2**14)
freq_ax.plot(f, sm_den, label='Filtered & Smoothed')

freq_ax.set_ylim(1.0E-19, 1.0E-4)
freq_ax.set_ylabel('Power Spectral Density [W/Hz]')
freq_ax.set_xlabel('Frequency [Hz]')
freq_ax.set_yscale('log')
freq_ax_legend = freq_ax.legend(loc='best')

for leg in freq_ax_legend.legendHandles:
    leg.set_linewidth(2.0)

fig_name = 'frequency'

freq_fig.savefig(fig_name + '.pdf', bbox_inches='tight')
run(['convert', '-density', dpi, fig_name + '.pdf', fig_name + '.png'])

# Run the Example

In [ ]:
# %matplotlib inline
x_lo = -0.04*1000.0
x_hi = 0.075*1000.0
cond_00_in_02_mm = ucr.Condition()
cond_00_in_02_mm.add_experiment(Path('00_in_02_mm_373K-1285t-100x-19-Jul-15-1620.txt'))
cond_00_in_02_mm.add_experiment(Path('00_in_02_mm_373K-1282t-100x-19-Jul-15-1626.txt'))
cond_00_in_02_mm.add_experiment(Path('00_in_02_mm_373K-1282t-100x-19-Jul-15-1633.txt'))
cond_00_in_02_mm.add_experiment(Path('00_in_02_mm_373K-1282t-100x-19-Jul-15-1640.txt'))
cond_00_in_02_mm.add_experiment(Path('00_in_02_mm_373K-1282t-100x-19-Jul-15-1646.txt'))
cond_00_in_02_mm.add_experiment(Path('NR_00_in_02_mm_373K-1278t-100x-19-Jul-15-1652.txt'))
cond_00_in_02_mm.create_volume_trace()
# cond_00_in_02_mm.compare_to_sim(run_reactive=True, run_nonreactive=True)

In [ ]:
fig_name = 'all-runs'
cond_00_in_02_mm.all_runs_figure.set_size_inches(14, 9)
cond_00_in_02_mm.all_runs_axis.set_xlim(x_lo, x_hi)
cond_00_in_02_mm.all_runs_figure.savefig(fig_name + '.pdf', bbox_inches='tight')
run(['convert', '-density', dpi, fig_name + '.pdf', fig_name + '.png'])

In [ ]:
fig_name = 'one-run'
exp_fig = cond_00_in_02_mm.reactive_experiments['00_in_02_mm_373K-1282t-100x-19-Jul-15-1633.txt'].exp_fig
exp_fig.set_size_inches(14, 9)
cond_00_in_02_mm.reactive_experiments['00_in_02_mm_373K-1282t-100x-19-Jul-15-1633.txt'].p_axis.set_xlim(x_lo, x_hi)
exp_fig.savefig(fig_name + '.pdf', bbox_inches='tight')
run(['convert', '-density', dpi, fig_name + '.pdf', fig_name + '.png'])

In [ ]:
fig_name = 'nonreactive-run'
cond_00_in_02_mm.nonreactive_figure.set_size_inches(14, 9)
cond_00_in_02_mm.nonreactive_axis.set_xlim(x_lo, x_hi)
cond_00_in_02_mm.nonreactive_figure.savefig(fig_name + '.pdf', bbox_inches='tight')
run(['convert', '-density', dpi, fig_name + '.pdf', fig_name + '.png'])

In [ ]:
fig_name = 'pressure-comparison'
cond_00_in_02_mm.pressure_comparison_figure.set_size_inches(14, 9)
cond_00_in_02_mm.pressure_comparison_axis.set_xlim(-36, -23)
cond_00_in_02_mm.pressure_comparison_axis.set_ylim(1.68, 1.85)
cond_00_in_02_mm.pressure_comparison_figure.savefig(fig_name + '.pdf', bbox_inches='tight')
run(['convert', '-density', dpi, fig_name + '.pdf', fig_name + '.png'])

In [ ]:
fig_name = 'simulation-comparison'
cond_00_in_02_mm.simulation_figure.set_size_inches(14, 9)
cond_00_in_02_mm.simulation_axis.set_xlim(x_lo, x_hi)
cond_00_in_02_mm.simulation_axis.legend(loc='best')
cond_00_in_02_mm.simulation_figure.savefig(fig_name + '.pdf', bbox_inches='tight')
run(['convert', '-density', dpi, fig_name + '.pdf', fig_name + '.png'])

# Draw a flowchart for program functions

In [ ]:
from graphviz import Digraph
from subprocess import run

dot = Digraph(filename='flowchart.dot', format='png', body=['newrank=true;'], node_attr={'style': 'filled', 'fillcolor': 'white'})

s3 = Digraph(node_attr={'shape': 'cds'})
s3.node('filename', 'Input Filename')
s3.node('yaml', 'volume-trace.yaml')
s3.node('tracesin', 'pressure.txt\nvolume.csv', width="1.5", height="0.75")
s3.node('input', 'Input')

dot.subgraph(s3)

s2 = Digraph(node_attr={'shape': 'rectangle'})
s2.node('condition', 'Condition')
s2.node('experiment', 'Experiment')
s2.node('fromtraces', 'VolumeFromPressure\nPressureFromVolume\nTemperatureFromPressure')
s2.node('simulation', 'Simulation')
s2.node('class', 'Class')

dot.subgraph(s2)

q1 = Digraph(node_attr={'shape': 'ellipse'})
q1.node('add_experiment()')
q1.node('compare_to_sim()')
q1.node('create_volume_trace()')
q1.node('function', 'Function')

dot.subgraph(q1)

s5 = Digraph(node_attr={'shape': 'parallelogram'})
s5.node('tau', '<Experimental &tau;<br />P<sub>c</sub>>', fixedsize='true', width='1.75')
s5.node('tracesout', 'pressure.txt\nvolume.csv', fixedsize='true', height='0.75', width='1.5')
s5.node('simout', '<Simulated &tau;<br />T<sub>c</sub>>', fixedsize='true', width='1.5', height='0.75')
s5.node('output', 'Output', shape='parallelogram')

dot.subgraph(s5)

s1 = Digraph(name='cluster0', graph_attr={'style': 'filled', 'fillcolor': 'lightgrey', 'label': 'Input'}, edge_attr={'style': 'invis'})
s1.edges([
        ('filename', 'yaml'),
        ('yaml', 'tracesin'),
    ])
dot.subgraph(s1)

f2 = Digraph(name='cluster3', graph_attr={'style': 'filled', 'fillcolor': 'white', 'label': 'Internal\nClasses'}, edge_attr={'style': 'invis'})
f2.edges([
        ('experiment', 'fromtraces'),
        ('fromtraces', 'simulation'),
    ])

dot.subgraph(f2)

f1 = Digraph(name='cluster2', graph_attr={'style': 'filled', 'fillcolor': 'violet', 'label': 'Outputs'}, edge_attr={'style': 'invis'})
f1.edges([
        ('tau', 'tracesout'),
        ('tracesout', 'simout')
    ])

dot.subgraph(f1)

s4 = Digraph(name='cluster1', graph_attr={'style': 'filled', 'fillcolor': 'orange', 'label': 'User Interface'})
s4.edges([
        ('condition', 'add_experiment()'),
        ('add_experiment()', 'create_volume_trace()'),
        ('create_volume_trace()', 'compare_to_sim()'),
    ])

dot.subgraph(s4)

e1 = Digraph(graph_attr={'rank': 'same'})
e1.edges([
        ('filename', 'add_experiment()'),
        ('add_experiment()', 'experiment'),
        ('experiment', 'tau'),
    ])

dot.subgraph(e1)

e2 = Digraph(graph_attr={'rank': 'same'})
e2.edges([
        ('yaml', 'create_volume_trace()'),
        ('create_volume_trace()', 'fromtraces'),
        ('fromtraces', 'tracesout'),
    ])

dot.subgraph(e2)

e3 = Digraph(graph_attr={'rank': 'same'})
e3.edges([
        ('tracesin', 'compare_to_sim()'),
        ('compare_to_sim()', 'simulation'),
        ('simulation', 'simout'),
    ])

dot.subgraph(e3)

g1 = Digraph(graph_attr={'rank': 'same', 'nodesep': '0.1'}, edge_attr={'style': 'invis'})
g1.edges([
        ('legend', 'class'),
        ('class', 'function'),
        ('function', 'input'),
        ('input', 'output'),
    ])
g1.node('legend', 'Legend:', shape='plaintext')

dot.subgraph(g1)

dot.edge('tracesin', 'legend', style='invis')

dot.save()
run(['dot', '-oflowchart.png', '-Tpng', 'flowchart.dot'])